In [6]:
import sys
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics


movie_reviews_data_folder = r'.\movie_review'
dataset = load_files(movie_reviews_data_folder, shuffle=False,encoding='utf8')
# print("n_samples: %d" % len(dataset.data))
RANDOM_STATE=940302
docs_train, docs_test, y_train, y_test = train_test_split(
    dataset.data, dataset.target, test_size=0.2, random_state=RANDOM_STATE,shuffle=True)
from sklearn.linear_model import SGDClassifier

# clf_pipe = Pipeline([
#     ('vect',CountVectorizer()),
#     ('tfidf',TfidfTransformer()),
#     ('clf',SGDClassifier())])

# clf_pipe.fit(docs_train,y_train)
# y_pre = clf_pipe.predict(docs_test)
# import numpy as np
# np.mean(y_pre==y_test)


In [67]:
# TASK: Build a grid search to find out whether unigrams or bigrams are
# more useful.
# Fit the pipeline on the training set using grid search for the parameters
from sklearn.naive_bayes import MultinomialNB
pipe_clf = Pipeline([('step1',TfidfVectorizer()), ('step2', MultinomialNB())])
params={'step1__ngram_range':[(1,1),(1,2)],
       'step2__alpha':(1e-1,1e-2,1e-3),
       }
gds = GridSearchCV(estimator=pipe_clf,param_grid=params)
gds = gds.fit(docs_train,y_train)
print(gds.cv_results_)
print('==========================')
print('最佳分数：{}'.format(gds.best_score_))                                  
for param_name in sorted(params.keys()):
    print("%s: %r" % (param_name, gds.best_params_[param_name]))

{'mean_fit_time': array([ 0.6846149 ,  0.65458536,  0.66853476,  2.96273073,  3.08042979,
        2.98036615]), 'std_fit_time': array([ 0.03108948,  0.00376383,  0.00857129,  0.1027057 ,  0.1181902 ,
        0.01864673]), 'mean_score_time': array([ 0.30867227,  0.28258952,  0.27658073,  0.72772272,  0.72240257,
        0.70943634]), 'std_score_time': array([ 0.02721284,  0.00892447,  0.00571772,  0.01502258,  0.01356086,
        0.02194635]), 'param_step1__ngram_range': masked_array(data = [(1, 1) (1, 1) (1, 1) (1, 2) (1, 2) (1, 2)],
             mask = [False False False False False False],
       fill_value = ?)
, 'param_step2__alpha': masked_array(data = [0.1 0.01 0.001 0.1 0.01 0.001],
             mask = [False False False False False False],
       fill_value = ?)
, 'params': [{'step1__ngram_range': (1, 1), 'step2__alpha': 0.1}, {'step1__ngram_range': (1, 1), 'step2__alpha': 0.01}, {'step1__ngram_range': (1, 1), 'step2__alpha': 0.001}, {'step1__ngram_range': (1, 2), 'step2__alpha

In [71]:
# TASK: Predict the outcome on the testing set and store it in a variable
# named y_predicted
y_predicted = gds.predict(docs_test)
print('混淆矩阵：\n{}'.format(metrics.confusion_matrix(y_predicted,y_test)))
# Print the classification report
print('---------------------------------------------------------------')
def pipecv_report(clf,params):
    print(clf.cv_results_)
    print('==========================')
    print('最佳分数：{}'.format(clf.best_score_))                                  
    for param_name in sorted(params.keys()):
        print("%s: %r" % (param_name, clf.best_params_[param_name]))
        
pipecv_report(gds,params)

混淆矩阵：
[[155  35]
 [ 37 173]]
---------------------------------------------------------------
{'mean_fit_time': array([ 0.6846149 ,  0.65458536,  0.66853476,  2.96273073,  3.08042979,
        2.98036615]), 'std_fit_time': array([ 0.03108948,  0.00376383,  0.00857129,  0.1027057 ,  0.1181902 ,
        0.01864673]), 'mean_score_time': array([ 0.30867227,  0.28258952,  0.27658073,  0.72772272,  0.72240257,
        0.70943634]), 'std_score_time': array([ 0.02721284,  0.00892447,  0.00571772,  0.01502258,  0.01356086,
        0.02194635]), 'param_step1__ngram_range': masked_array(data = [(1, 1) (1, 1) (1, 1) (1, 2) (1, 2) (1, 2)],
             mask = [False False False False False False],
       fill_value = ?)
, 'param_step2__alpha': masked_array(data = [0.1 0.01 0.001 0.1 0.01 0.001],
             mask = [False False False False False False],
       fill_value = ?)
, 'params': [{'step1__ngram_range': (1, 1), 'step2__alpha': 0.1}, {'step1__ngram_range': (1, 1), 'step2__alpha': 0.01}, {'step

In [7]:
#运行pipe2 要reload dataset
# pipe_2 = Pipeline([('step1',TfidfVectorizer()), ('step2', SGDClassifier(random_state=940302,
#                                            max_iter=200, tol=None))])

pipe_2 = Pipeline([('step1',TfidfVectorizer()), ('step2',LinearSVC())])
pipe_2_params={'step1__ngram_range':[(1,1),(1,2)],
                'step1__max_df': (0.6,0.7),
              'step2__C':(1.0,0.5)
       }
gds_2 = GridSearchCV(estimator=pipe_2,param_grid=pipe_2_params)
gds_2 = gds_2.fit(docs_train,y_train)
print(gds_2.cv_results_)
print('==========================')


{'mean_fit_time': array([0.72811357, 0.68135198, 3.08693345, 3.00028515, 0.69230858,
       0.6706934 , 3.04959941, 2.97755941]), 'std_fit_time': array([0.03591782, 0.00753077, 0.06320311, 0.01958779, 0.01801354,
       0.00826088, 0.02621912, 0.02640809]), 'mean_score_time': array([0.31064677, 0.28937125, 0.72003667, 0.72669872, 0.28664978,
       0.29333266, 0.71466899, 0.72943727]), 'std_score_time': array([0.04054738, 0.00817614, 0.00866117, 0.02623995, 0.01643976,
       0.00822236, 0.01236596, 0.01364171]), 'param_step1__max_df': masked_array(data=[0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_step1__ngram_range': masked_array(data=[(1, 1), (1, 1), (1, 2), (1, 2), (1, 1), (1, 1), (1, 2),
                   (1, 2)],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_step2__C

In [8]:
print('最佳分数：{}'.format(gds_2.best_score_))                                  
for param_name in sorted(pipe_2_params.keys()):
    print("%s: %r" % (param_name, gds_2.best_params_[param_name]))
y_predicted = gds_2.predict(docs_test)
print(metrics.classification_report(y_test, y_predicted,target_names=dataset.target_names))
print(metrics.accuracy_score(y_test,y_predicted))

最佳分数：0.84875
step1__max_df: 0.7
step1__ngram_range: (1, 2)
step2__C: 1.0
             precision    recall  f1-score   support

        neg       0.87      0.84      0.85       192
        pos       0.86      0.88      0.87       208

avg / total       0.86      0.86      0.86       400

0.8625
